Load libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Define the transformations to apply to the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define the dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:13<00:00, 12672588.19it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:

# Define the models
models_dict = {
    'VGG-16': models.vgg16(pretrained=True),
    'ResNet-18': models.resnet18(pretrained=True),
    'GoogLeNet': models.googlenet(pretrained=True),
    'DenseNet-121': models.densenet121(pretrained=True),
    'MobileNetV2': models.mobilenet_v2(pretrained=True)
}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:01<00:00, 296MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. 

In [7]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# # Define the optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Function to calculate accuracy
def get_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = (correct / total)*100
    return accuracy

# Training loop


for model_name, model in models_dict.items():
    print(f"Training {model_name}...")
    model.to(device)  # Move model to GPU if available
    model.train()  # Set model to training mode
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(5):
      running_loss = 0.0
        
      for inputs, labels in trainloader:
          inputs = inputs.to(device)
          labels = labels.to(device)
          
          optimizer.zero_grad()
          
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          
          running_loss += loss.item()
      
      epoch_loss = running_loss / len(trainloader)
      print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")


    # Evaluation
    model.eval()  # Set model to evaluation mode
    total_accuracy = 0.0
    
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            accuracy = get_accuracy(outputs, labels)
            total_accuracy += accuracy
        
    avg_accuracy = total_accuracy / len(testloader)
    print(f"Accuracy on test set for {model_name}: {avg_accuracy:.2f}%")
    print("----------------------------")


Training VGG-16...
Epoch 1: Loss = 0.5488
Epoch 2: Loss = 0.5124
Epoch 3: Loss = 0.4843
Epoch 4: Loss = 0.4555
Epoch 5: Loss = 0.4415
Accuracy on test set for VGG-16: 83.89%
----------------------------
Training ResNet-18...
Epoch 1: Loss = 1.0136
Epoch 2: Loss = 0.8822
Epoch 3: Loss = 0.8048
Epoch 4: Loss = 0.7491
Epoch 5: Loss = 0.7126
Accuracy on test set for ResNet-18: 74.43%
----------------------------
Training GoogLeNet...
Epoch 1: Loss = 3.1591
Epoch 2: Loss = 1.4556
Epoch 3: Loss = 1.2576
Epoch 4: Loss = 1.1227
Epoch 5: Loss = 1.0409
Accuracy on test set for GoogLeNet: 66.37%
----------------------------
Training DenseNet-121...
Epoch 1: Loss = 2.0853
Epoch 2: Loss = 0.8764
Epoch 3: Loss = 0.7407
Epoch 4: Loss = 0.6508
Epoch 5: Loss = 0.5923
Accuracy on test set for DenseNet-121: 78.01%
----------------------------
Training MobileNetV2...
Epoch 1: Loss = 2.1631
Epoch 2: Loss = 1.0543
Epoch 3: Loss = 0.9175
Epoch 4: Loss = 0.8330
Epoch 5: Loss = 0.7802
Accuracy on test set for 

| Architecture   | Accuracy     |
|----------------|--------------|
| GoogLeNet      | 66.37%       |
| VGG-16         | 83.89%       |
| ResNet-18      | 77.43%       |
| DenseNet-121   | 78.01%       |
| MobileNetV2    | 73.18%       |
